##  ‧₊˚🎄✩ ₊˚🦌⊹♡ Advent of Code 2024 ‧₊˚🎄✩ ₊˚🦌⊹♡

In [1]:
import itertools
import functools
import collections
import numpy as np
import pandas as pd

----------

Day 1

In [11]:
def solve(i: int):
    return 1

solve(10)

1